### Plan of Attack:

1. Get list of new songs after a specific date
2. Get data needed for those songs
3. Create parquet table
4. Save to S3

In [ ]:
# import spotipy
# import os
# %run './credentials/settings.py'
# from spotipy.oauth2 import SpotifyOAuth
# scope = "user-library-read"

# def user_auth(scope=scope):
#     SPOTIPY_CLIENT_ID = os.getenv("SPOTIPY_CLIENT_ID")
#     SPOTIPY_CLIENT_SECRET = os.getenv("SPOTIPY_CLIENT_SECRET")
#     SPOTIPY_REDIRECT_URI = os.getenv('SPOTIPY_REDIRECT_URI_READ')
#     print()
#     sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))
#     return sp
# sp = user_auth()

In [1]:
!pip install spotipy

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


In [22]:
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy
import os
import pandas as pd
import numpy as np
%run '../../credentials/settings.py'
from collections import defaultdict

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=os.getenv("SPOTIPY_CLIENT_ID"),
                                                           client_secret=os.getenv("SPOTIPY_CLIENT_SECRET")))

In [4]:
new_releases = sp.new_releases(country='US')

In [10]:
len(new_releases['albums']['items'])

20

In [11]:
new_releases['albums']['items'][0].keys()

dict_keys(['album_type', 'artists', 'available_markets', 'external_urls', 'href', 'id', 'images', 'name', 'release_date', 'release_date_precision', 'total_tracks', 'type', 'uri'])

In [16]:
new_releases['albums']['items'][0]['artists']

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/56oDRnqbIiwx4mymNEv7dS'},
  'href': 'https://api.spotify.com/v1/artists/56oDRnqbIiwx4mymNEv7dS',
  'id': '56oDRnqbIiwx4mymNEv7dS',
  'name': 'Lizzo',
  'type': 'artist',
  'uri': 'spotify:artist:56oDRnqbIiwx4mymNEv7dS'},
 {'external_urls': {'spotify': 'https://open.spotify.com/artist/4kYSro6naA4h99UJvo89HB'},
  'href': 'https://api.spotify.com/v1/artists/4kYSro6naA4h99UJvo89HB',
  'id': '4kYSro6naA4h99UJvo89HB',
  'name': 'Cardi B',
  'type': 'artist',
  'uri': 'spotify:artist:4kYSro6naA4h99UJvo89HB'}]

In [26]:
artist_lst = new_releases['albums']['items'][0]['artists']
artist_ids = str([x['id'] for x in artist_lst])
artist_names = [x['name'] for x in artist_lst]
print(artist_ids, artist_names,sep='\n')

['56oDRnqbIiwx4mymNEv7dS', '4kYSro6naA4h99UJvo89HB']
['Lizzo', 'Cardi B']


In [27]:
artist_ids

"['56oDRnqbIiwx4mymNEv7dS', '4kYSro6naA4h99UJvo89HB']"

### list items we want from each album, will need to figure out how to get track ids from each album (even if an artist releases a single their is an "album" associated)
- release_date
- total_tracks
- id
- name
- artist(s)
- artist ids

In [39]:
def album_metadata(album_items):
    album_data = {}
    album_data['id'] = []
    album_data['release_date'] = []
    album_data['total_tracks']  = []
    album_data['artist_id_1'] = []
    album_data['artist_ids'] = []
    album_data['artist_names'] = []
    
    for album_item in album_items:
    
        # map 1:1 data for each record
        album_data['id'].append(album_item['id'])
        album_data['release_date'].append(album_item['release_date'])
        album_data['total_tracks'].append(album_item['total_tracks'])

        # get artist(s) data
        artist_lst = album_item['artists']
        artist_ids = [x['id'] for x in artist_lst]
        album_data['artist_id_1'].append(artist_ids[0])
        album_data['artist_ids'].append(str(artist_ids))
        album_data['artist_names'].append(str([x['name'] for x in artist_lst]))
    
    return pd.DataFrame.from_dict(album_data,orient='columns')

In [40]:
#test it
album_df = album_metadata(new_releases['albums']['items'])
album_df

,id,release_date,total_tracks,artist_id_1,artist_ids,artist_names
0,7jMo8dhMIorHoLlzTisjJy,2021-08-13,1,56oDRnqbIiwx4mymNEv7dS,"['56oDRnqbIiwx4mymNEv7dS', '4kYSro6naA4h99UJvo...","['Lizzo', 'Cardi B']"
1,2TvfE8CY37OQIPVGcWYpEA,2021-08-12,1,2o5jDhtHVPhrJdv3cEQ99Z,"['2o5jDhtHVPhrJdv3cEQ99Z', '790FomKkXshlbRYZFt...","['Tiësto', 'KAROL G']"
2,5D8Rdb09BkmHscEGSWAlA6,2021-08-13,1,3PhoLpVuITZKcymswpck5b,"['3PhoLpVuITZKcymswpck5b', '6M2wZ9GZgrQXHCFfjv...","['Elton John', 'Dua Lipa', 'PNAU']"
3,00fcDjvEq4elj756TESO0c,2021-08-13,1,78rUTD7y6Cy67W1RVzYs7t,['78rUTD7y6Cy67W1RVzYs7t'],['PinkPantheress']
4,1qMFjBarjO2xD15BwXZguD,2021-08-13,11,0C0XlULifJtAgn6ZNCW2eu,['0C0XlULifJtAgn6ZNCW2eu'],['The Killers']
5,6vqNFOfIgqe0l2Ui47YSyO,2021-08-13,6,2cFrymmkijnjDg9SS92EPM,['2cFrymmkijnjDg9SS92EPM'],['blackbear']
6,4ih8LPpqdYPezAMaPAHJYT,2021-08-07,1,46SHBwWsqBkxI7EeeBEQG7,"['46SHBwWsqBkxI7EeeBEQG7', '45TgXXqMDdF8BkjA83...","['Kodak Black', 'Rod Wave']"
7,3k4lUXBZxjUGFFQvy8PsHn,2021-08-12,1,6TIYQ3jFPwQSRmorSezPxX,['6TIYQ3jFPwQSRmorSezPxX'],['Machine Gun Kelly']
8,4pWo1MFBvVAfhjL4aPZFaD,2021-08-13,1,1vyhD5VmyZ7KMfW5gqLgo5,['1vyhD5VmyZ7KMfW5gqLgo5'],['J Balvin']
9,7L8IHgiomfEpxOm61vgPTm,2021-08-13,12,7z5WFjZAIYejWy0NI5lv4T,['7z5WFjZAIYejWy0NI5lv4T'],['Dan + Shay']


### figure out how to get track IDs from each of the above

In [ ]:
def find_song(name, year):
  
    """
    This function returns a dataframe with data for a song given the name and release year.
    The function uses Spotipy to fetch audio features and metadata for the specified song.
    
    """
    
    song_data = defaultdict()
    results = sp.search(q= 'track: {} year: {}'.format(name,
                                                       year), limit=1)
    if results['tracks']['items'] == []:
        return None
    
    results = results['tracks']['items'][0]

    track_id = results['id']
    audio_features = sp.audio_features(track_id)[0]
    
    song_data['name'] = [name]
    song_data['year'] = [year]
    song_data['explicit'] = [int(results['explicit'])]
    song_data['duration_ms'] = [results['duration_ms']]
    song_data['popularity'] = [results['popularity']]
    
    for key, value in audio_features.items():
        song_data[key] = value
    
    return pd.DataFrame(song_data)
    